# Deep Learning based Recommender System

Traditionally, recommender systems are based on methods such as clustering, nearest neighbor and matrix factorization. However, in recent years, deep learning has yielded tremendous success across multiple domains, from image recognition to natural language processing. Recommender systems have also benefited from deep learning's success. In fact, today's state-of-the-art recommender systems such as those at [YouTube](https://research.google/pubs/pub45530/) and [Amazon](https://www.amazon.science/the-history-of-amazons-recommendation-algorithm) are powered by complex deep learning systems, and less so on traditional methods.

# Why this notebook?

While reading through the many useful notebooks here that covers the basics of recommender systems using traditional methods such as matrix factorization, I noticed that there is a lack of tutorials that cover deep learning based recommender systems. In this notebook, we'll go through the following:

* How to create your own deep learning based recommender system using PyTorch Lightning
* The difference between implicit and explicit feedback for recommender systems
* How to train-test split a dataset for training recommender systems without introducing biases and data leakages 
* Metrics for evaluating recommender systems (hint: accuracy or RMSE is not appropriate!)

# Building Recommender Systems using Implicit Feedback

Before we build our model, it is important to understand the distinction between *implicit* and *explicit* feedback in the context of recommender systems, and why modern recommender systems are built on implicit feedback.

### Explicit Feedback

In the context of recommender systems, explicit feedback are **direct** and **quantitative** data collected from users. For example, Amazon allows users to rate purchased items on a scale of 1-10. These ratings are provided directly from users, and the scale allows Amazon to quantify user preference. Another example of explicit feedback includes the thumbs up button on YouTube, which captures users' explicit preference (i.e. like or dislike) of a particular video.

However, the problem with explicit feedback is that they are rare. If you think about it, when was the last time you clicked the like button on a YouTube video, or rated your online purchases? Chances are, the amount of videos you watch on YouTube is far greater than the amount of videos that you have explicitly rated.

### Implicit Feedback

On the other hand, implicit feedback are collected indirectly from user **interactions**, and they act as a proxy for user preference. For example. videos that you watch on YouTube are used as implicit feedback to tailor recommendations for you, even if you don't rate the videos explicitly. Another example of implicit feedback includes the items that you have browsed on Amazon, which are used to suggest other similar items for you.

The advantage of implicit feedback is that it is abundant. Recommender systems built using implicit feedback also allows us to tailor recommendations in real time, with every click and interaction. Today, online recommender systems are built using implicit feedback, which allows the system to tune its recommendation in real-time, with every user interaction.

However, implicit feedback has its shortcomings as well. Unlike explicit feedback, every interaction is assumed to be positive and we are unable to capture negative preference from users. How then do we capture negative feedback? One technique that can be applied is negative sampling, which we will go through in a later section.

# Data Preprocessing

Before we start building and training our model, let's do some preprocessing to get the data in the required format.

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

np.random.seed(123)

First, we import the ratings dataset.

In [2]:
ratings = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv', 
                      parse_dates=['timestamp'])

In order to keep memory usage manageable within Kaggle's kernel, we will only use data from 30% of the users in this dataset. Let's randomly select 30% of the users and only use data from the selected users.

In [3]:
rand_userIds = np.random.choice(ratings['userId'].unique(), 
                                size=int(len(ratings['userId'].unique())*0.3), 
                                replace=False)

ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]

print('There are {} rows of data from {} users'.format(len(ratings), len(rand_userIds)))

There are 6027314 rows of data from 41547 users


In [4]:
ratings.sample(5)

,userId,movieId,rating,timestamp
3840312,26182,3704,4.0,2007-01-31 21:56:52
7608731,52439,3365,4.0,2004-03-21 08:02:56
19363634,134060,1027,3.0,2003-07-15 22:43:45
17181947,118860,2629,1.0,2007-11-29 21:27:08
9344779,64638,4723,2.0,2001-09-10 20:11:41


After filtering the dataset, there are now 6,027,314  rows of data from 41,547 users. Each row in the dataframe corresponds to a movie review made by a single user. 

### Train-test split

Along with the rating, there is also a `timestamp` column that shows the date and time the review was submitted. Using the `timestamp` column, we will implement our train-test split strategy using the leave-one-out methodology. For each user, the most recent review is used as the test set (i.e. leave one out), while the rest will be used as training data .

To illustrate this, the movies reviewed by user 39,849 is shown below. The last movie reviewed by the user is the 2014 hit movie Guardians of The Galaxy. We'll use this movie as the testing data for this user, and use the rest of the reviewed movies as training data.

![](https://i.imgur.com/oNJnLqU.png)
> **Movie posters from themoviedb.org (free to use)**
> 



This train-test split strategy is often used when training and evaluating recommender systems. Doing a random split would not be fair, as we could potentially be using a user's recent reviews for training and earlier reviews for testing. This introduces data leakage with a look-ahead bias, and the performance of the trained model would not be generalizable to real-world performance.

The code below will split our ratings dataset into a train and test set using the leave-one-out methodology.

In [5]:
ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'] \
                                .rank(method='first', ascending=False)

train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

# drop columns that we no longer need
train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

### Converting the dataset into an implicit feedback dataset

As discussed earlier, we will train a recommender system using implicit feedback. However, the MovieLens dataset that we're using is based on explicit feedback. To convert this dataset into an implicit feedback dataset, we'll simply binarize the ratings such that they are are '1' (i.e. positive class). The value of '1' represents that the user has interacted with the item.

It is important to note that using implicit feedback reframes the problem that our recommender is trying to solve. Instead of trying to predict movie ratings (when using explicit feedback), we are trying to predict whether the user will interact (i.e. click/buy/watch) with each movie, with the aim of presenting to users the movies with the highest interaction likelihood.






In [6]:
train_ratings.loc[:, 'rating'] = 1

train_ratings.sample(5)

,userId,movieId,rating
3411906,23263,5481,1
1815983,12245,5464,1
16198592,112109,6,1
13914487,96124,1247,1
1445807,9790,1690,1


We do have a problem now though. After binarizing our dataset, we see that every sample in the dataset now belongs to the positive class. However we also require negative samples to train our models, to indicate movies that the user has not interacted with. We assume that such movies are those that the user are not interested in - even though this is a sweeping assumption that may not be true, it usually works out rather well in practice.

The code below generates 4 negative samples for each row of data. In other words, the ratio of negative to positive samples is 4:1. This ratio is chosen arbitrarily but I found that it works rather well (feel free to find the best ratio yourself!)

In [7]:
# Get a list of all movie IDs
all_movieIds = ratings['movieId'].unique()

# Placeholders that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train_ratings['userId'], train_ratings['movieId']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_movieIds) 
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

Great! We now have the data in the format required by our model. Before we move on, let's define a PyTorch Dataset to facilitate training. The class below simply encapsulates the code we have written above into a PyTorch Dataset class.

In [8]:
class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
    
    """

    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['userId'], ratings['movieId']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

# Our model - Neural Collaborative Filtering (NCF)

While there are many deep learning based architecture for recommendation systems, I find that the framework proposed by [He et al.](https://arxiv.org/abs/1708.05031) is the most straightforward and it is simple enough to be implemented in a tutorial such as this.

### User Embeddings

Before we dive into the architecture of the model, let's familiarize ourselves with the concept of embeddings. An embedding is a low-dimensional space that captures the relationship of vectors from a higher dimensional space. To better understand this concept, let's take a closer look at user embeddings. 

Imagine that we want to represent our users according to their preference for two genres of movies - action and romance movies. Let the first dimension be how much the user likes action movies, and the second dimension be how much the user likes romance movies.

![](https://i.imgur.com/XENzqXq.png)

Now, assume that Bob is our first user. Bob likes action movies but isn't a fan of romance movies. To represent Bob as a two dimensional vector, we place him in the graph according to his preference.

![](https://i.imgur.com/rSStTCj.png)

Our next user is Joe. Joe is a huge fan of both action and romance movies. We represent Joe using a two dimensional vector just like Bob.

![](https://i.imgur.com/gmmkrEU.png)

This two dimensional space is known as an embedding. Essentially, the embedding reduces our users such that they can be represented in a meaningful manner in a lower dimensional space. In this embedding, users with similar movie preferences are placed near to each other, and vice versa. 

![](https://i.imgur.com/9s9Z7JT.png)

Of course, we are not restricted to using just 2 dimensions to represent our users. We can use an arbitrary number of dimensions to represent our users. A larger number of dimensions would allow us to capture the traits of each user more accurately, at the cost of model complexity. In our code, we'll use 8 dimensions (which we will see later).

### Learned Embeddings

Similarly, we will use a separate item embedding layer to represent the traits of the items (i.e. movies) in a lower dimensional space. 

You might be wondering, how can we learn the weights of the embedding layer, such that it provides an accurate representation of users and items? In our previous example, we used Bob and Joe's preference for action and romance movies to manually create our embedding. Is there a way to learn such preferences automatically? 

The answer is **Collaborative Filtering** - by using the ratings dataset, we can identify similar users and movies, creating user and item embeddings learned from existing ratings.

### Model Architecture

Now that we have a better understanding of embeddings, we are ready to define the model architecture. As you'll see, the user and item embeddings are key to the model.

<!-- ![NCF](https://i.imgur.com/EZh1HHf.png)
 -->
 
Let's walk through the model architecture using the following training sample:

| userId | movieID | interacted |
|-|-|-|
| 3 | 1 | 1 |

 
![](https://i.imgur.com/cNWbIce.png)


The inputs to the model are the one-hot encoded user and item vector for `userId = 3` and `movieId = 1`. Because this is a positive sample (movie actually rated by the user), the true label (`interacted`) is 1.

The user input vector and item input vector are fed to the user embedding and item embedding respectively, which results in a smaller, denser user and item vectors. 

The embedded user and item vectors are concatenated before passing through a series of fully connected layers, which maps the concatenated embeddings into a prediction vector as output. Finally, we apply a `Sigmoid` function to obtain the most probable class. In the example above, the most probable class is 1 (positive class), since 0.8 > 0.2.


Now, let's define this NCF model using PyTorch Lightning!

In [9]:
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=512, num_workers=4)

We instantiate the NCF model using the class that we have defined above.

In [10]:
num_users = ratings['userId'].max()+1
num_items = ratings['movieId'].max()+1

all_movieIds = ratings['movieId'].unique()

model = NCF(num_users, num_items, train_ratings, all_movieIds)

Let's train our NCF model for 5 epochs using the GPU. Notice that we are using the argument `reload_dataloaders_every_epoch=True`. This creates a new randomly chosen set of negative samples for each epoch, which ensures that our model is not biased by the selection of negative samples.

Note: One advantage of [PyTorch Lightning](https://github.com/PyTorchLightning/pytorch-lightning) over vanilla PyTorch is that you don't need to write your own boiler plate training code. Notice how the [Trainer](https://pytorch-lightning.readthedocs.io/en/latest/trainer.html) class allows us to train our model with just a few lines of code. 

In [11]:
trainer = pl.Trainer(max_epochs=5, gpus=1, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 1 M   
1 | item_embedding | Embedding | 1 M   
2 | fc1            | Linear    | 1 K   
3 | fc2            | Linear    | 2 K   
4 | output         | Linear    | 33    


Saving latest checkpoint..


1

# Evaluating our Recommender System

Now that our model is trained, we are ready to evaluate it using the test data. In traditional Machine Learning projects, we evaluate our models using metrics such as Accuracy (for classification problems) and RMSE (for regression problems). However, such metrics are too simplistic for evaluating recommender systems.

To design a good metric for evaluating recommender systems, we need to first understand how modern recommender systems are used. 

Looking at Netflix, we see a list of recommendations like the one below:

![](https://i.imgur.com/5QRWcYy.jpg)

Similarly, Amazon uses a list of recommendations:

![](https://i.imgur.com/XZZ2Ni8.png)

The key here is that we don't need the user to interact on *every* single item in the list of recommendations. Instead, we just need the user to interact with at least one item on the list - as long as the user does that, the recommendations have worked.

To simulate this, let's run the following evaluation protocol to generate a list of 10 recommended items for each user.

* For each user, randomly select 99 items that the user **has not interacted with**
* Combine these 99 items with the test item (the actual item that the user interacted with). We now have 100 items.
* Run the model on these 100 items, and rank them according to their predicted probabilities
* Select the top 10 items from the list of 100 items. If the test item is present within the top 10 items, then we say that this is a hit.
* Repeat the process for all users. The Hit Ratio is then the average hits.

This evaluation protocol is known as **Hit Ratio @ 10**, and it is commonly used to evaluate recommender systems. 

### Hit Ratio @ 10 

Now, let's evaluate our model using the described protocol.

In [12]:
# User-item pairs for testing
test_user_item_set = set(zip(test_ratings['userId'], test_ratings['movieId']))

# Dict of all items that are interacted with by each user
user_interacted_items = ratings.groupby('userId')['movieId'].apply(list).to_dict()

hits = []
for (u,i) in tqdm(test_user_item_set):
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))


The Hit Ratio @ 10 is 0.86


We got a pretty good Hit Ratio @ 10 score! To put this into context, what this means is that 86% of the users were recommended the actual item (among a list of 10 items) that they eventually interacted with. Not bad!

# What's Next?

I hope that this has been an useful introduction to creating a deep learning based recommender systems. To learn more, I recommend the following resources:
* [Wide & Deep Learning - Model introduced by Google for Recommender Systems](https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html)
* [Recommenders library by Microsoft - Best practices for Recommender Systems](https://github.com/microsoft/recommenders)
* [Deep Learning based Recommender Systems - Useful survey paper](https://arxiv.org/pdf/1707.07435.pdf)